In [10]:
# Modules

In [11]:
import pandas as pd
import numpy as np

In [12]:
# Data import

In [13]:
test_data = pd.read_csv('./data/test.csv').set_index('id')
test_data.head()

,commentaire
id,
0,Bon jeu!on peut &eacute;voluer tout au long du...
1,Mon fils de 2 ans a eu ce train pour noël. Il ...
2,Training card game sympa mais sans plus on en ...
3,Super livre je le recommande les les recettes ...
4,"J'AI BEAUCOUP AIME CE FILM, UN DESSIN ANIME TR..."


In [14]:
train_data = pd.read_csv('./data/train.csv').set_index('id')
train_data.head()

,commentaire,evaluation
id,,
0,Ayant obtenu un succ&egrave;s massif des deux ...,positif
1,FOR THE FIRST TIME IN OVER 20 YEARS – AN EXPER...,positif
2,Une édition hors de prix avec une bande son fr...,negatif
3,Je suis étonnée par les commentaires péjoratif...,positif
4,Ca s'est bien passé et puis voilà ! De plus le...,positif


In [15]:
train_data.describe()

,commentaire,evaluation
count,60000,60000
unique,59688,2
top,"Envoi Parfait et soigné, je suis très content ...",positif
freq,9,42363


In [16]:
# Traitement des données

In [34]:
import re
import nltk.corpus
from nltk.corpus import stopwords
from django.utils.text import unescape_entities
# nltk.download('stopwords')

def clean_text(df, text_field, new_text_field_name):
    stop = stopwords.words('french') + stopwords.words('english')
    char_list = ['&', ';']

    # Majuscule
    df[new_text_field_name] = df[text_field].str.lower()
    # Les mots en avec des & et ; dedans
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: unescape_entities(elem))
    # Accent
    df[new_text_field_name] = df[new_text_field_name].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    # <br />
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r'<br />', ' ', elem))
    # Caractere spéciaux
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r'(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])', '', elem))
    # Nombre
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r'\d+', '', elem))
    # Double espace
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r'\s+', ' ', elem))
    # Mot de liaison
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: ' '.join([word for word in elem.split() if word not in (stop)]))
    
    return df

In [35]:
train_data_clean = clean_text(train_data, 'commentaire', 'commentaire_clean')
test_data_clean = clean_text(test_data, 'commentaire', 'commentaire_clean')
train_data_clean.head()

,commentaire,evaluation,commentaire_clean
id,,,
0,Ayant obtenu un succ&egrave;s massif des deux ...,positif,obtenu succes massif deux cotes latlantique bl...
1,FOR THE FIRST TIME IN OVER 20 YEARS – AN EXPER...,positif,first time years expert authoritative work rio...
2,Une édition hors de prix avec une bande son fr...,negatif,edition hors prix bande francaise nest hd enco...
3,Je suis étonnée par les commentaires péjoratif...,positif,etonnee commentaires pejoratif film effet jai ...
4,Ca s'est bien passé et puis voilà ! De plus le...,positif,ca sest bien passe puis voila plus cd coldplay...


In [19]:
X_test = np.array(test_data_clean['commentaire_clean'])
X, y = np.array(train_data_clean['commentaire_clean']), np.array(train_data_clean['evaluation'])

In [37]:
print(train_data_clean['commentaire'].iloc[25])
print()
print(train_data_clean['commentaire_clean'].iloc[25])

bonjour,<br />il amis 5 jours au lieu de 24h comme pr&eacute;vu de plus la pochette &eacute;tait toute cass&eacute;e heureusement le dvd n'avait rien

bonjour amis jours lieu h comme prevu plus pochette etait toute cassee heureusement dvd navait rien


In [39]:
# Code

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=69420)

# X_train
X_train_counts = CountVectorizer().fit_transform(X_train)
X_train_tfidf = TfidfTransformer().fit_transform(X_train_counts)

# X_valid
X_valid_counts = CountVectorizer().fit_transform(X_valid)
X_valid_tfidf = TfidfTransformer().fit_transform(X_valid_counts)

In [41]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)
clf.score(X_train_tfidf, y_train)

0.7631666666666667

In [43]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
clf.score(X_train_tfidf, y_train)

0.9105625